In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import re

df_train = pd.read_csv('train.csv')
# df_train['reg_date'] = pd.to_datetime(df_train['reg_date'], format='%d-%b-%Y', errors='coerce')

# current_year = 2024
# current_month = 7

# df_train['original_index'] = df_train.index

df_train['original_depreciation'] = df_train['depreciation']
df_train['original_omv'] = df_train['omv']
df_train['original_arf'] = df_train['arf']

def replace_make(row):
    make = row['title'].split()[0].strip().lower() if pd.isnull(row['make']) else row['make'].lower()
    if make == 'maybach':
        return 'mercedes-benz'
    else:
        return make

df_train['make'] = df_train.apply(replace_make, axis=1)

def is_coe_extended(title):
    if re.findall(r'\(.*COE.*\)',title):
        return True
    else:
        return False

df_train['COE_extended'] = np.vectorize(is_coe_extended)(df_train['title'])

# df_train['COE_extended'].value_counts()

df_train['reg_date'] = pd.to_datetime(df_train['reg_date'])

# 定义计算成熟日期的函数
def maturity_date(coe_extended, title, reg_date):
    if coe_extended:
        # 如果标题中包含类似“MM/YYYY”格式的日期
        if re.findall(r'\d{2}/\d{4}', title):
            maturity_date = pd.to_datetime(re.findall(r'\d{2}/\d{4}', title)[0], format='%m/%Y').date()
        else:
            # 如果标题中没有日期信息，延长20年
            maturity_date = (reg_date + pd.DateOffset(years=20)).date()
    else:
        # 如果没有延长 COE，则默认10年
        maturity_date = (reg_date + pd.DateOffset(years=10)).date()
    
    return maturity_date

# 使用矢量化函数来创建成熟日期列
df_train['maturity_date'] = np.vectorize(maturity_date)(df_train['COE_extended'], df_train['title'], df_train['reg_date'])

def years_left(coe_extended, reg_date, maturity_date):
    # 计算到成熟日期的天数并转换为年
    years_left = (pd.to_datetime(maturity_date) - pd.to_datetime('2024-06-20')).days / 365.25
    if years_left < -1:
        return years_left % 10
    elif years_left < 1:
        return 1 + years_left
    return round(years_left, 1)

# 使用矢量化函数创建剩余年份列
df_train['remaining_years_of_coe'] = np.vectorize(years_left)(df_train['COE_extended'], df_train['reg_date'], df_train['maturity_date'])

df_train.drop(columns=['COE_extended', 'maturity_date'], inplace=True)
# df_train['years_left'].value_counts()

# df_train['maturity_date'].value_counts()

# df_train['remaining_years_of_coe'] = 10 - (current_year - df_train['reg_date'].dt.year + (current_month - df_train['reg_date'].dt.month) / 12) % 10

def calculate_arf(omv):
    if omv <= 20000:
        return omv
    elif 20000 < omv <= 40000:
        return 20000 + 1.4 * (omv - 20000)
    elif 40000 < omv <= 60000:
        return 20000 + 1.4 * 20000 + 1.9 * (omv - 40000)
    elif 60000 < omv <= 80000:
        return 20000 + 1.4 * 20000 + 1.9 * 20000 + 2.5 * (omv - 60000)
    else:
        return 20000 + 1.4 * 20000 + 1.9 * 20000 + 2.5 * 20000 + 3.2 * (omv - 80000)

# Function to calculate OMV based on ARF
def calculate_omv(arf):
    if arf <= 20000:
        return arf
    elif 20000 < arf <= 48000:
        return 20000 + (arf - 20000) / 1.4
    elif 48000 < arf <= 86000:
        return 40000 + (arf - 48000) / 1.9
    elif 86000 < arf <= 136000:
        return 60000 + (arf - 86000) / 2.5
    else:
        return 80000 + (arf - 136000) / 3.2

missing_depreciation = df_train['depreciation'].isnull().sum()
missing_omv = df_train['omv'].isnull().sum()
missing_arf = df_train['arf'].isnull().sum()

# Print the results
print(f'Missing values in depreciation: {missing_depreciation}')
print(f'Missing values in omv: {missing_omv}')
print(f'Missing values in arf: {missing_arf}')

# Fill missing ARF values using OMV
df_train['arf'] = df_train.apply(
    lambda row: calculate_arf(row['omv']) if pd.isnull(row['arf']) and pd.notnull(row['omv']) else row['arf'], axis=1
)

# Fill missing OMV values using ARF
df_train['omv'] = df_train.apply(
    lambda row: calculate_omv(row['arf']) if pd.isnull(row['omv']) and pd.notnull(row['arf']) else row['omv'], axis=1
)

missing_depreciation = df_train['depreciation'].isnull().sum()
missing_omv = df_train['omv'].isnull().sum()
missing_arf = df_train['arf'].isnull().sum()

# Print the results
print(f'Missing values in depreciation: {missing_depreciation}')
print(f'Missing values in omv: {missing_omv}')
print(f'Missing values in arf: {missing_arf}')


def fill_missing_by_title_median(df, column):
    # Extract the first three words of the title
    df['title_first_three'] = df['title'].apply(lambda x: ' '.join(x.split()[:3]))

    for idx in df[df[column].isnull()].index:
        current_title_first_three = df.loc[idx, 'title_first_three']
        # Find entries with the same title_first_three
        similar_titles = df[df['title_first_three'] == current_title_first_three]
        if not similar_titles[column].dropna().empty:
            # Calculate the median and fill the missing value
            median_value = similar_titles[column].median()
            df.loc[idx, column] = median_value

    # Drop the temporary title_first_three column
    df.drop(columns=['title_first_three'], inplace=True)
    
    return df

missing_depreciation = df_train['depreciation'].isnull().sum()
missing_omv = df_train['omv'].isnull().sum()
missing_arf = df_train['arf'].isnull().sum()

# Print the results
print(f'Missing values in depreciation: {missing_depreciation}')
print(f'Missing values in omv: {missing_omv}')
print(f'Missing values in arf: {missing_arf}')

# Apply the function to fill missing values
for column in ['depreciation', 'omv', 'arf']:
    df_train = fill_missing_by_title_median(df_train, column)

missing_depreciation = df_train['depreciation'].isnull().sum()
missing_omv = df_train['omv'].isnull().sum()
missing_arf = df_train['arf'].isnull().sum()

# Print the results
print(f'Missing values in depreciation: {missing_depreciation}')
print(f'Missing values in omv: {missing_omv}')
print(f'Missing values in arf: {missing_arf}')

def fill_missing_by_model_mean(df, column):
    def fill_value(group):
        if group.notna().sum() > 0:
            return group.fillna(group.mean())
        else:
            return group  # Return as is if all values are NaN
    df[column] = df.groupby('model')[column].transform(fill_value)
    return df

# Apply the function to fill missing values
for column in ['depreciation', 'omv', 'arf']:
    df_train = fill_missing_by_model_mean(df_train, column)

# Display the dataframe to verify
missing_depreciation = df_train['depreciation'].isnull().sum()
missing_omv = df_train['omv'].isnull().sum()
missing_arf = df_train['arf'].isnull().sum()

# Print the results
print(f'Missing values in depreciation: {missing_depreciation}')
print(f'Missing values in omv: {missing_omv}')
print(f'Missing values in arf: {missing_arf}')

# Find rows where any of the specified columns are still null
null_models = df_train[df_train[['depreciation', 'omv', 'arf']].isnull().any(axis=1)]

for column in ['depreciation', 'omv', 'arf']:
    df_train[column] = df_train[column].fillna(df_train[column].mean())


def classify_category(category):
    if 'parf car' in category.lower():
        return 'parf car'
    elif 'coe car' in category.lower():
        return 'coe car'
    else:
        return 'coe car'

# Apply the classification function
df_train['car_classification'] = df_train['category'].apply(classify_category)

model_price_stats = df_train.groupby('model')['price'].agg(['max', 'min']).reset_index()
model_price_stats.columns = ['model', 'max_model_price', 'min_model_price']
# 用test前，需要先跑一次train，然后将上面两行要注释掉

make_price_stats = df_train.groupby('make')['price'].agg(['max', 'min']).reset_index()
make_price_stats.columns = ['make', 'max_make_price', 'min_make_price']
# 用test前，需要先跑一次train，然后将上面两行要注释掉

df_train = df_train.merge(model_price_stats, on='model', how='left')

df_train = df_train.merge(make_price_stats, on='make', how='left')

df_train['max_price'] = df_train['max_model_price'].combine_first(df_train['max_make_price'])
df_train['min_price'] = df_train['min_model_price'].combine_first(df_train['min_make_price'])


current_year = 2024
df_train['vehicle_age_at_dereg'] = current_year - df_train['manufactured']

# Function to calculate the PARF rebate
# def calculate_parf_rebate(row):
#     if row['vehicle_age_at_dereg'] < 5:
#         rebate = 0.75 * row['arf']
#     elif 5 <= row['vehicle_age_at_dereg'] < 6:
#         rebate = 0.70 * row['arf']
#     elif 6 <= row['vehicle_age_at_dereg'] < 7:
#         rebate = 0.65 * row['arf']
#     elif 7 <= row['vehicle_age_at_dereg'] < 8:
#         rebate = 0.60 * row['arf']
#     elif 8 <= row['vehicle_age_at_dereg'] < 9:
#         rebate = 0.55 * row['arf']
#     elif 9 <= row['vehicle_age_at_dereg'] < 10:
#         rebate = 0.50 * row['arf']
#     else:
#         rebate = 0  # No rebate for vehicles older than 10 years
    
#     # Apply the cap if the car was registered on or after 15 February 2023
#     if row['reg_date'] >= pd.Timestamp('2023-02-15'):
#         rebate = min(rebate, 60000)
    
#     return rebate

# # Apply the PARF rebate calculation
# df_train['parf_rebate'] = df_train.apply(calculate_parf_rebate, axis=1)

# Check for any remaining missing values
missing_values_after = df_train[['depreciation', 'omv', 'arf', 'max_model_price', 'min_model_price', 'max_make_price', 'min_make_price', 'max_price', 'min_price', 'manufactured']].isnull().sum()
print(f"Missing values after filling: {missing_values_after}")

# 问题，test数据集表现不佳，可能是有一些model只有train有，需考虑是否使用make来代替

def calculate_list_price(row):
    if pd.isnull(row['remaining_years_of_coe']):
        return None
        
    if row['reg_date'] >= pd.Timestamp('2013-03-01'):
        min_parf_value = 0.5 * row['arf']
    elif row['reg_date'] >= pd.Timestamp('2008-03-01'):
        min_parf_value = 0.5 * row['omv'] if row['omv'] < row['arf'] else 0.5 * row['arf']
    else:
        min_parf_value = 0.55 * row['omv'] if row['omv'] < row['arf'] else 0.5 * row['arf']

    list_price = row['depreciation'] * row['remaining_years_of_coe']
    coe_price = row['remaining_years_of_coe'] * row['coe'] / 10
    # parf_price = row['parf_rebate']
    parf_price = min_parf_value
    # if pd.notnull(row['dereg_value']):
      # list_price += row['dereg_value']
    # else:
    if row['car_classification'] == 'parf car':  # assuming there's a column 'coe_vehicle' indicating COE status
        list_price += parf_price
      # list_price += coe_price


    # if row['car_classification'] == 'parf car':
    #     list_price = row['depreciation'] * row['remaining_years_of_coe'] + row['remaining_years_of_coe'] * row['coe'] / 10
    # else:
    #     list_price = row['depreciation'] * row['remaining_years_of_coe']
    
    # Adjust for COE vehicle
        # else:
        #     list_price += row['parf_rebate']
    
    # return (list_price - 450) * 0.967
    return list_price

df_train['indicative_price'] = df_train.apply(calculate_list_price, axis=1)

bias = 380
weight = 0.943



# Adjust the predicted prices
def adjust_predicted_price(row):
    if pd.isnull(row['max_price']) or pd.isnull(row['min_price']):
        return row['indicative_price']
    # if pd.notnull(row['dereg_value']) and row['indicative_price'] < 1.5 * row['dereg_value']:
    #     return 1.5 * row['dereg_value']
    if row['indicative_price'] > row['max_price']:
        return row['max_price']
    elif row['indicative_price'] < row['min_price']:
        return row['min_price']
    else:
        return row['indicative_price']

df_train['indicative_price'] = df_train.apply(adjust_predicted_price, axis=1)


# def calculate_used_car_price(row):
#     used_car_price = row['omv'] * (0.15 + 0.85 * (1 - row['vehicle_age_at_dereg'] / 10)) + row['remaining_years_of_coe'] * row['coe']
#     return used_car_price
  
# df_train['indicative_price'] = df_train.apply(calculate_used_car_price, axis=1)

# Drop the temporary max_price and min_price columns if you don't need them
# df_train.drop(columns=['max_price', 'min_price'], inplace=True)

print(df_train[['model', 'indicative_price', 'max_price', 'min_price']].head())


df_train.drop(columns=['listing_id', 'description', 'original_reg_date', 'reg_date', 'fuel_type', 'opc_scheme', 'lifespan', 'eco_category', 'features', 'accessories', 'original_depreciation', 'original_omv', 'original_arf', 'max_model_price', 'min_model_price', 'max_make_price', 'min_make_price'], inplace=True)
df_train.info()

D:\TEMP\ipykernel_5628\1251424638.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_train['reg_date'] = pd.to_datetime(df_train['reg_date'])


Missing values in depreciation: 201
Missing values in omv: 29
Missing values in arf: 65
Missing values in depreciation: 201
Missing values in omv: 29
Missing values in arf: 29
Missing values in depreciation: 201
Missing values in omv: 29
Missing values in arf: 29
Missing values in depreciation: 26
Missing values in omv: 26
Missing values in arf: 26
Missing values in depreciation: 9
Missing values in omv: 13
Missing values in arf: 13
Missing values after filling: depreciation        0
omv                 0
arf                 0
max_model_price    77
min_model_price    77
max_make_price      2
min_make_price      2
max_price           2
min_price           2
manufactured        3
parf_rebate         0
dtype: int64
    model  indicative_price  max_price  min_price
0   vezel      37408.189254   165800.0    14800.0
1       3      36036.000000   168000.0     3800.0
2  cooper     152200.500000   268800.0     7777.0
3    vios      76286.500000   121800.0     2500.0
4       3      26414.500000 

In [28]:
def fill_missing_by_make_median(df, column):
    def fill_value(group):
        if group.notna().sum() > 0:
            return group.fillna(group.median())
        else:
            return group  # Return as is if all values are NaN
    df[column] = df.groupby('model')[column].transform(fill_value)
    return df

for column in ['manufactured', 'curb_weight', 'power', 'engine_cap', 'no_of_owners',  'road_tax', 'dereg_value', 'mileage']:
  df_train = fill_missing_by_model_median(df_train, column)
df_train.isnull().sum()

df_train['vehicle_age_at_dereg'] = current_year - df_train['manufactured']

def fill_missing_by_make_median(df, column):
    def fill_value(group):
        if group.notna().sum() > 0:
            return group.fillna(group.median())
        else:
            return group  # Return as is if all values are NaN
    df[column] = df.groupby('make')[column].transform(fill_value)
    return df

for column in ['curb_weight', 'power', 'engine_cap', 'no_of_owners',  'road_tax', 'dereg_value', 'mileage']:
  df_train = fill_missing_by_make_median(df_train, column)
df_train.isnull().sum()


for column in ['curb_weight', 'power', 'engine_cap', 'no_of_owners',  'road_tax', 'dereg_value', 'mileage']:
  df_train[column] = df_train[column].fillna(df_train[column].median())
df_train.isnull().sum()
df_train.info()
df_train.to_csv('test_cleaned.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   title                   10000 non-null  object 
 1   make                    10000 non-null  object 
 2   model                   10000 non-null  object 
 3   manufactured            10000 non-null  float64
 4   type_of_vehicle         10000 non-null  object 
 5   category                10000 non-null  object 
 6   transmission            10000 non-null  object 
 7   curb_weight             10000 non-null  float64
 8   power                   10000 non-null  float64
 9   engine_cap              10000 non-null  float64
 10  no_of_owners            10000 non-null  float64
 11  depreciation            10000 non-null  float64
 12  coe                     10000 non-null  int64  
 13  road_tax                10000 non-null  float64
 14  dereg_value             10000 non-null 

In [16]:
import pandas as pd
from datetime import datetime
import numpy as np
import re
# from sklearn.ensemble import RandomForestRegressor
# import lightgbm as lgb
# from sklearn.ensemble import GradientBoostingRegressor
# from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
# from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
# Step 5: Encode Categorical Variables
categorical_features = ['make', 'model', 'type_of_vehicle', 'category', 'transmission', 'car_classification']

numerical_features = ['curb_weight', 'power', 'engine_cap', 'no_of_owners', 'depreciation', 'road_tax', 'dereg_value', 'mileage', 'omv', 'arf', 'indicative_price', 'remaining_years_of_coe', 'max_price', 'min_price', 'vehicle_age_at_dereg']

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
    ])

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

df_train = pd.read_csv('train_cleaned.csv')

X = df_train.drop(columns=['title', 'manufactured', 'price'])
y = df_train['price']
X_preprocessed = pipeline.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

model = Ridge(alpha=1.0)
model.fit(X_train, y_train)
# model.fit(X_preprocessed, y)

y_test_pred = model.predict(X_test)
# y_pred = model.predict(X_preprocessed)

mae_test = mean_absolute_error(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)
# mae_test = mean_absolute_error(y, y_pred)
# mse_test = mean_squared_error(y, y_pred)
# rmse_test = np.sqrt(mse_test)

print(f'Test Set Performance:')
print(f'Mean Absolute Error: {mae_test}')
print(f'Mean Squared Error: {mse_test}')
print(f'Root Mean Squared Error: {rmse_test}')

Test Set Performance:
Mean Absolute Error: 3036.5195341249937
Mean Squared Error: 144779723.39188203
Root Mean Squared Error: 12032.444614120692


In [15]:
df_test = pd.read_csv('test_cleaned.csv')
X_pred = df_test.drop(columns=['title', 'manufactured'])
X_pred_preprocessed = pipeline.transform(X_pred)
y_res = model.predict(X_pred_preprocessed)

make_predictions = pd.DataFrame({
    'Id': df_test.index,
    'Predicted': y_res
})

make_predictions.to_csv('Lasso.csv', index=False)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001681 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3816
[LightGBM] [Info] Number of data points in the train set: 20000, number of used features: 356
[LightGBM] [Info] Start training from score 115632.681850
Test Set Performance:
Mean Absolute Error: 3485.022388617732
Mean Squared Error: 288943707.8723001
Root Mean Squared Error: 16998.344268554512

XGBoost Regressor: 
Test Set Performance:
Mean Absolute Error: 3796.2905301757814
Mean Squared Error: 404668613.5832261
Root Mean Squared Error: 20116.376750877036

GradientBoostingRegressor: 
Test Set Performance:
Mean Absolute Error: 3333.8404578557806
Mean Squared Error: 169268452.2040934
Root Mean Squared Error: 13010.320987742516

Ridge Regression:
Test Set Performance:
Mean Absolute Error: 3036.5195341249937
Mean Squared Error: 144779723.39188203
Root Mean Squared Error: 12032.444614120692

Lasso Regression:
Test Set Performance:
Mean Absolute Error: 3324.9121472571214
Mean Squared Error: 154769866.71283126
Root Mean Squared Error: 12440.653789605723